In [ ]:
%pip install torch
%pip install git+https://github.com/illuin-tech/colpali
%pip install pdf2image pypdf

In [ ]:
import os

import torch
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import (
    process_images,
    process_queries,
)
from pdf2image import convert_from_path
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoProcessor


In [ ]:
device = "mps"

# Load model
model_name = "vidore/colpali"
token = os.environ.get("HF_TOKEN")
model = ColPali.from_pretrained(
    "google/paligemma-3b-mix-448", torch_dtype=torch.float32, device_map="mps", token=token).eval()

model.load_adapter(model_name)
processor = AutoProcessor.from_pretrained(model_name, token = token)

In [ ]:


files = ["climate_youth_magazine.pdf"]

images = []
for f in files:
    images.extend(convert_from_path(f))

# run inference - docs
dataloader = DataLoader(
    images,
    batch_size=4,
    shuffle=False,
    collate_fn=lambda x: process_images(processor, x),
)

print(model.device)



In [ ]:
for batch_doc in tqdm(dataloader):
    with torch.no_grad():
        batch_doc = {k: v.to(device) for k, v in batch_doc.items()}
        embeddings_doc = model(**batch_doc)
    ds.extend(list(torch.unbind(embeddings_doc.to("cpu"))))
print(f"Uploaded and converted {len(images)} pages")
